## Imports

In [1]:
import pandas as pd
import numpy as np

## Read data

In [2]:
df = pd.read_csv("../data/interim/loans_int.csv")

/home/dangj/PycharmProjects/fca-takehome-test/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data_dict = pd.read_csv('../references/data-dictionary.csv')

## Objectives

- Understand the data
- Check data for completeness na's and cells with whitespace and resolve
- Check object data type and resolve

Overall picture:

- Want to answer: How to predict if an individual will default or pay their full loan?
- Model to be used as a tool to support analysts => simple model to use and understand
- Worth assessing the monetary value of model
- Part of IRB process to build a PD model

- Metrics to use are Acc, AUC/ROC, Recall

- Recall is important because allows business decisions to be made and how much capital to hold for emergency/unforeseen purposes dictated by IRB => investment opportunities would be missed out on if recall was low

## Data dictionary - data understanding

- When someone applies for a loan we would not know their loan status! So this would be a type of data leakage and should not be present in the dataset 


- Currency is in USD $


- Can drop post code for district as they represent same thing


- Use loan status as a binary predictor for default or not, combine {Late, Charged Off, Default}
    - Reason for this is that if you don't pay your bills in time => Late => Default => Charged Off
    - Combine Late due to compliance with IRB (https://www.bankofengland.co.uk/-/media/boe/files/prudential-regulation/supervisory-statement/2019/ss1113update-march2019) as we have the option to do so. Also combine to help minimise how imbalanced the data is
    
    
- Remove Ongoing samples, as the information there tells us about current loans, while we care about loans which have defaulted and fully paid, while this is information loss, how much information we lose is reasonable as Ongoing dataset accounts for only 2\% of the entire dataset


- Remove loan status as this feature is not available when trying to predict a loan application (data leakage), 
- same with issue date, this is something we get given that a loan was granted and understand the risks that comes with it, which is what we're trying to reduce i.e PD


In [4]:
df.shape

(237436, 32)

In [5]:
df.head()

,account_id,installment,loan_amount,interest_rate,term,purpose,issue_date,description,title,home_ownership,...,credit_card_usage,credit_card_balance,total_current_balance,nr_accounts,loan_status,amount_payed,year,district,postcode_district,credit_score
0,125968,829.10,25000.0,11.89,36 months,debt_consolidation,Aug-2009,Due to a lack of personal finance education an...,Debt consolidation for on-time payer,RENT,...,52.1,28854.0,NaN,42.0,Fully Paid,29324.320000,2009,Liverpool,L69,710.699725
1,128479,40.50,1200.0,13.11,36 months,debt_consolidation,Mar-2010,"If funded, I would use this loan consolidate t...",zxcvb,OWN,...,40.4,2584.0,NaN,31.0,Fully Paid,1457.310000,2010,Redbridge,IG9,416.086929
2,128650,366.86,10800.0,13.57,36 months,debt_consolidation,Nov-2009,I currently have a personal loan with Citifina...,Nicolechr1978,RENT,...,25.6,3511.0,NaN,40.0,Fully Paid,13195.270000,2009,Rugby,CV21,354.228832
3,129758,264.11,7200.0,19.05,36 months,debt_consolidation,Aug-2012,"Credit cards are out of here, I am tired of be...",caminijio,RENT,...,90.1,3874.0,154930.0,25.0,Fully Paid,9100.804663,2012,West Lindsey,DN21,697.214163
4,130240,102.92,3000.0,14.26,36 months,credit_card,Sep-2009,I am seeking to refinance a credit account whi...,Rejecting new cardmember agreement,MORTGAGE,...,39.5,4740.0,NaN,23.0,Fully Paid,3703.380000,2009,South Derbyshire,DE11,799.428700


In [6]:
df.loan_status.value_counts()

Fully Paid          206202
Charged Off          14956
Late (> 90 days)      9298
Ongoing               5761
Default               1219
Name: loan_status, dtype: int64

In [7]:
for idx in range(data_dict.shape[0]):
    variable = data_dict.iloc[idx,0]
    description = data_dict.iloc[idx,1]
    print(f'{variable}:\n{description}\n\n')

annual_income:
The self-reported annual income provided by the borrower during registration.


delinquency_2y:
The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years.


description:
Loan description provided by the borrower.


debt_to_income:
A ratio calculated using the borrower's total monthly debt payments on the total debt obligations, excluding mortgage and the requested loan, divided by the borrower’s self-reported monthly income.


earliest_credit_line:
The month the borrower's earliest reported credit line was opened.


employment_length:
Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.


job_title:
The job title supplied by the borrower when applying for the loan.


home_ownership:
The home ownership status provided by the borrower during registration. The values are: RENT, OWN, MORTGAGE, OTHER.


inquiries_6m:
The number of inquiries in past 6

## Data completeness

In [8]:
df_nan = [col for col in df.columns if df[col].isnull().any()==True]
print(df_nan)

['description', 'title', 'employment_length', 'job_title', 'last_record_months', 'last_delinquency_months', 'last_derog_months', 'credit_card_usage', 'total_current_balance', 'district']


## Data types

In [9]:
df_obj_type = [col for col in df.columns if df[col].dtypes=='object']
print(df_obj_type)

['term', 'purpose', 'issue_date', 'description', 'title', 'home_ownership', 'annual_income', 'employment_length', 'job_title', 'earliest_credit_line', 'loan_status', 'district', 'postcode_district']


## Resolve completeness

- Need to resolve variables with nan in columns
- Impute total_current_balance with the IQR split by region/district this way we remove potential outliers in the amounts shown by descriptive statistics


In [10]:
dict_cols_resolve = {}
dropped = []
impute = []
sample_drop = []

for col in df_nan:
    num_nan = df[[col]].isna().sum()
    total = df.shape[0]
    nan_pct = (num_nan/total * 100).values[0] 
    
    if nan_pct >= 50:
        dropped.append(col)
    elif nan_pct < 50 and nan_pct >=20:
        impute.append(col)
    else:
        sample_drop.append(col)

dict_cols_resolve['Drop'] = dropped
dict_cols_resolve['Impute'] = impute
dict_cols_resolve['Drop Samples'] = sample_drop

print(dict_cols_resolve)

{'Drop': ['description', 'last_record_months', 'last_delinquency_months', 'last_derog_months'], 'Impute': ['total_current_balance'], 'Drop Samples': ['title', 'employment_length', 'job_title', 'credit_card_usage', 'district']}


In [11]:
for col in dict_cols_resolve['Drop Samples']:
    print(f'{col}\n{df[col].isna().sum()}\n')

title
12

employment_length
9177

job_title
12601

credit_card_usage
173

district
888



In [12]:
df['job_title'].value_counts()

Teacher                              1953
Manager                              1912
Registered Nurse                      865
Supervisor                            860
RN                                    844
                                     ... 
Catch 35 Chicago                        1
MHC OPERATING LIMITED PARTNERSHIP       1
Hawkeye                                 1
APPTIS                                  1
AVMS                                    1
Name: job_title, Length: 123190, dtype: int64

## Resolve Datatypes

- issue_date to datetime format
- merge description to purpose, some are grouped as credit card but are actually debt consolidation
- simplify purpose down to about 4 groups (debt_consolidation, other, leisure, work_and_education)

In [13]:
df['purpose'].value_counts()

debt_consolidation    139509
credit_card            47820
home_improvement       14260
other                  12913
major_purchase          5920
small_business          3957
car                     3416
medical                 2601
moving                  1834
wedding                 1742
house                   1514
vacation                1485
educational              236
renewable_energy         229
Name: purpose, dtype: int64

In [14]:
#df[(df['description'].str.contains('paying off')==True)].head()
#df[df.index==21].description.values #example of debt consolidation
df[df['purpose']=='renewable_energy'].description.value_counts()

  Borrower added on 05/27/10 > It is a personal reason that I am helping out a close family member<br/> Borrower added on 05/27/10 > This loan is to help out a family member in need of financial support.<br/> Borrower added on 05/27/10 > This is loan to help out an immediate family member .<br/> Borrower added on 06/02/10 > This loan will be repaid as soon as possible, possibly within 90 days<br/> Borrower added on 06/08/10 > I am willing to go for a lower amount, say $10,000<br/> Borrower added on 06/08/10 > I will gladly accept whatever is offered to me with great thanks.<br/>                                                                                                                                                                                                                     1
  Borrower added on 10/22/10 > Loan will provide update to Main Bathroom, updating heating systems in home and payoff of a higher intrest Credit card.  <br/>I work for a fortune 80 company for 7 years